# Mask R-CNN - Train cell nucleus Dataset


This notebook shows how to train Mask R-CNN implemented on coco on your own dataset. I trained the model to segment cell nucleus objects in an image. You'd need a GPU, because the network backbone is a Resnet101, which would be too slow to train on a CPU.The code is execuatble on google colaboratory GPU.  On google colab you can start to get okay-ish results in a few minutes, and good results in less than an hour.



In [1]:
from google.colab import drive
drive.mount('/content/drive')

%pip install wget
%pip install pycocotools

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
import json
import pandas as pd
import wget
from zipfile import ZipFile
import tensorflow as tf
import datetime

from pycocotools.coco import COCO
import skimage.io as io
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# from keras.callbacks import Callback

from keras.layers import Layer

from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize

# download dataset from github or mount from google drive
# https://github.com/chaozhong2010/HRSID

# Directory of images to run detection on
DATA_DIR = os.path.join('drive/MyDrive/HRSID/HRSID_png/')
DATA_TYPE = 'train_test2017'
annotations_file = '{}/annotations/{}.json'.format(DATA_DIR, DATA_TYPE)
IMAGE_DIR = '{}/images/'.format(DATA_DIR)

dpi = 192 # monitor resolution

# Initialise the COCO api for instance annotations
coco = COCO(annotations_file)

# Load the categories in a variable
category_ids = coco.getCatIds(catNms=['ship'])
categories = coco.loadCats(category_ids)

print(categories)

# Get all images containing the above Category IDs
img_ids = coco.getImgIds(catIds=category_ids)
DATASET_SIZE = len(img_ids)
print("\nNumber of images containing ships in the %s annotations file:" % DATA_TYPE, DATASET_SIZE)

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

ImportError: numpy.core.multiarray failed to import

In [ ]:
if not os.path.exists('config.py'):
  print("Please upload the config.py to your google colab")
  from google.colab import files
  src = list(files.upload().values())[0]
  open('config.py','wb').write(src)
import config

if not os.path.exists('utils.py'):
  print("Please upload the utils.py to your google colab")
  from google.colab import files
  src = list(files.upload().values())[0]
  print(src)
  open('utils.py','wb').write(src)
import utils

if not os.path.exists('model.py'):
  print("Please upload the model.py to your google colab")
  from google.colab import files
  src = list(files.upload().values())[0]
  open('model.py','wb').write(src)
import model

if not os.path.exists('visualize.py'):
  print("Please upload the visualize.py to your google colab")
  from google.colab import files
  src = list(files.upload().values())[0]
  open('visualize.py','wb').write(src)
import visualize

from config import Config
import utils
import model as modellib
import visualize
from model import log

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline

# Root directory of the project
ROOT_DIR = os.getcwd()

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Directory to save logs and model checkpoints, if not provided
# through the command line argument --logs
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")

## Calculate MAP, precision, recall for each image

In [ ]:
# class EvalImage():
#   def __init__(self,dataset,model,cfg):
#     self.dataset = dataset
#     self.model   = model
#     self.cfg     = cfg

 
 
#   def evaluate_model(self , len = 50):
#     APs = list()
#     precisions_dict = {}
#     recall_dict     = {}
#     for index,image_id in enumerate(self.dataset.image_ids):
#       if(index > len):
#          break; 
#       # load image, bounding boxes and masks for the image id
#       image, image_meta, gt_class_id, gt_bbox, gt_mask = modellib.load_image_gt(self.dataset, self.cfg,image_id, use_mini_mask=False)
#       # convert pixel values (e.g. center)
#       #scaled_image = modellib.mold_image(image, self.cfg)
#       # convert image into one sample
#       sample = np.expand_dims(image, 0)
#      # print(len(image))
#       # make prediction
#       yhat = self.model.detect(sample, verbose=1)
#       # extract results for first sample
#       r = yhat[0]
#       # calculate statistics, including AP
#       AP, precisions, recalls, _ = compute_ap(gt_bbox, gt_class_id, gt_mask, r["rois"], r["class_ids"], r["scores"], r['masks'])
#       precisions_dict[image_id] = np.mean(precisions)
#       recall_dict[image_id] = np.mean(recalls)
#       # store
#       APs.append(AP)

#     # calculate the mean AP across all images
#     mAP = np.mean(APs)
#     return mAP,precisions_dict,recall_dict

## Configurations

In [ ]:
class ShapesConfig(Config):
    """Configuration for training on the dataset.
    Derives from the base Config class and overrides values specific
    to the dataset.
    """
    # Give the configuration a recognizable name
    NAME = "shapes"

    # Train on 1 GPU and 1 images per GPU. We can put multiple images on each
    # GPU. Batch size is (GPUs * images/GPU).
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # background + nucleus

    # Use small images for faster training. Set the limits of the small side
    # the large side, and that determines the image shape.
    IMAGE_MIN_DIM = 832
    IMAGE_MAX_DIM = 832
    IMAGE_SHAPE=[832, 832, 3]

    # Use smaller anchors because our image and objects are small
    RPN_ANCHOR_SCALES = (8, 16, 64, 128, 256)  # anchor side in pixels

    # Aim to allow ROI sampling to pick 33% positive ROIs.
    TRAIN_ROIS_PER_IMAGE = 800

    # set number of epoch
    STEPS_PER_EPOCH = 200

    # set validation steps 
    VALIDATION_STEPS = 50

    # The height and width are those of the image unless use_mini_mask is True,
    # in which case they are defined in MINI_MASK_SHAPE.
    # If enabled, resizes instance masks to a smaller size to reduce
    # memory load. Recommended when using high-resolution images.
    USE_MINI_MASK = False
    MINI_MASK_SHAPE = (IMAGE_SHAPE[0], IMAGE_SHAPE[1])  # (height, width) of the mini-mask

    
config = ShapesConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  896
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  896
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [896 896   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE             

## Notebook Preferences

In [ ]:
def get_ax(rows=1, cols=1, size=8):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

## Dataset

Create a synthetic dataset

Extend the Dataset class and add a method to load the shapes dataset, `load_shapes()`, and override the following methods:

* load_image()
* load_mask()
* image_reference()

In [ ]:
class ShipsDataset(utils.Dataset):
    def load_coco(self, dataset_dir, subset, class_ids=None,
                  class_map=None, return_coco=False, auto_download=False):
        """Load a subset of the COCO dataset.
        dataset_dir: The root directory of the COCO dataset.
        subset: What to load (train, val, minival, valminusminival)
        year: What dataset year to load (2014, 2017) as a string, not an integer
        class_ids: If provided, only loads images that have the given classes.
        class_map: TODO: Not implemented yet. Supports maping classes from
            different datasets to the same class ID.
        return_coco: If True, returns the COCO object.
        auto_download: Automatically download and unzip MS-COCO images and annotations
        """

        if auto_download is True:
            self.auto_download(dataset_dir, subset)
        print(dataset_dir)
        print(subset)
        coco = COCO(annotations_file)
        if subset == "minival" or subset == "valminusminival":
            subset = "val"
        image_dir = IMAGE_DIR

        # Load all classes or a subset?
        if not class_ids:
            # All classes
            class_ids = sorted(coco.getCatIds())

        # All images or a subset?
        if class_ids:
            image_ids = []
            for id in class_ids:
                image_ids.extend(list(coco.getImgIds(catIds=[id])))
            # Remove duplicates
            image_ids = list(set(image_ids))
        else:
            # All images
            image_ids = list(coco.imgs.keys())

        # Add classes
        for i in class_ids:
            self.add_class("coco", i, coco.loadCats(i)[0]["name"])

        # Add images
        print(image_dir)
        coco.imgs[0]['file_name']
        for i in image_ids:
            self.add_image(
                "coco", image_id=i,
                path=os.path.join(image_dir, coco.imgs[i]['file_name']),
                file_name=coco.imgs[i]['file_name'],
                width=config.IMAGE_SHAPE[0],
                height=config.IMAGE_SHAPE[1],
                annotations=coco.loadAnns(coco.getAnnIds(
                    imgIds=[i], catIds=class_ids, iscrowd=None)))
        if return_coco:
            return coco

    def auto_download(self, dataDir, dataType, dataYear):
        """Download the COCO dataset/annotations if requested.
        dataDir: The root directory of the COCO dataset.
        dataType: What to load (train, val, minival, valminusminival)
        dataYear: What dataset year to load (2014, 2017) as a string, not an integer
        Note:
            For 2014, use "train", "val", "minival", or "valminusminival"
            For 2017, only "train" and "val" annotations are available
        """

        # Setup paths and file names
        if dataType == "minival" or dataType == "valminusminival":
            imgDir = "{}/{}{}".format(dataDir, "val", dataYear)
            imgZipFile = "{}/{}{}.zip".format(dataDir, "val", dataYear)
            imgURL = "http://images.cocodataset.org/zips/{}{}.zip".format("val", dataYear)
        else:
            imgDir = "{}/{}{}".format(dataDir, dataType, dataYear)
            imgZipFile = "{}/{}{}.zip".format(dataDir, dataType, dataYear)
            imgURL = "http://images.cocodataset.org/zips/{}{}.zip".format(dataType, dataYear)
        # print("Image paths:"); print(imgDir); print(imgZipFile); print(imgURL)

        # Create main folder if it doesn't exist yet
        if not os.path.exists(dataDir):
            os.makedirs(dataDir)

        # Download images if not available locally
        if not os.path.exists(imgDir):
            os.makedirs(imgDir)
            print("Downloading images to " + imgZipFile + " ...")
            with urllib.request.urlopen(imgURL) as resp, open(imgZipFile, 'wb') as out:
                shutil.copyfileobj(resp, out)
            print("... done downloading.")
            print("Unzipping " + imgZipFile)
            with zipfile.ZipFile(imgZipFile, "r") as zip_ref:
                zip_ref.extractall(dataDir)
            print("... done unzipping")
        print("Will use images in " + imgDir)

        # Setup annotations data paths
        annDir = "{}/annotations".format(dataDir)
        if dataType == "minival":
            annZipFile = "{}/instances_minival2014.json.zip".format(dataDir)
            annFile = "{}/instances_minival2014.json".format(annDir)
            annURL = "https://dl.dropboxusercontent.com/s/o43o90bna78omob/instances_minival2014.json.zip?dl=0"
            unZipDir = annDir
        elif dataType == "valminusminival":
            annZipFile = "{}/instances_valminusminival2014.json.zip".format(dataDir)
            annFile = "{}/instances_valminusminival2014.json".format(annDir)
            annURL = "https://dl.dropboxusercontent.com/s/s3tw5zcg7395368/instances_valminusminival2014.json.zip?dl=0"
            unZipDir = annDir
        else:
            annZipFile = "{}/annotations_trainval{}.zip".format(dataDir, dataYear)
            annFile = "{}/instances_{}{}.json".format(annDir, dataType, dataYear)
            annURL = "http://images.cocodataset.org/annotations/annotations_trainval{}.zip".format(dataYear)
            unZipDir = dataDir
        # print("Annotations paths:"); print(annDir); print(annFile); print(annZipFile); print(annURL)

        # Download annotations if not available locally
        if not os.path.exists(annDir):
            os.makedirs(annDir)
        if not os.path.exists(annFile):
            if not os.path.exists(annZipFile):
                print("Downloading zipped annotations to " + annZipFile + " ...")
                with urllib.request.urlopen(annURL) as resp, open(annZipFile, 'wb') as out:
                    shutil.copyfileobj(resp, out)
                print("... done downloading.")
            print("Unzipping " + annZipFile)
            with zipfile.ZipFile(annZipFile, "r") as zip_ref:
                zip_ref.extractall(unZipDir)
            print("... done unzipping")
        print("Will use annotations in " + annFile)

    def load_mask(self, image_id):
        """Load instance masks for the given image.
        Different datasets use different ways to store masks. This
        function converts the different mask format to one format
        in the form of a bitmap [height, width, instances].
        Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        # If not a COCO image, delegate to parent class.
        image_info = self.image_info[image_id]
        if image_info["source"] != "coco":
            return super(CocoDataset, self).load_mask(image_id)

        instance_masks = []
        class_ids = []
        # print(self.image_info[image_id]["annotations"])
        annotations = self.image_info[image_id]["annotations"]
        # Build mask of shape [height, width, instance_count] and list
        # of class IDs that correspond to each channel of the mask.
        for annotation in annotations:
            class_id = self.map_source_class_id(
                "coco.{}".format(annotation['category_id']))
            if class_id:
                # print(config.MINI_MASK_SHAPE)
                empty_mask = np.zeros((800,800))
                # print(empty_mask.shape)
                m = np.maximum(coco.annToMask(annotation), empty_mask)
                # # Some objects are so small that they're less than 1 pixel area
                # and end up rounded out. Skip those objects.
                # if m.max() < 1:
                #     continue
                # Is it a crowd? If so, use a negative class ID.
                # if annotation['iscrowd']:
                    # Use negative class ID for crowds
                    # class_id *= -1
                    # For crowd masks, annToMask() sometimes returns a mask
                    # smaller than the given dimensions. If so, resize it.
                if m.shape[0] != image_info["height"] or m.shape[1] != image_info["width"]:
                    m = resize(m, (832, 832)) # resize the ground truth to the new image size
                instance_masks.append(m)
                class_ids.append(class_id)

        # Pack instance masks into an array
        if class_ids:
            mask = np.stack(instance_masks, axis=2).astype(np.bool)
            class_ids = np.array(class_ids, dtype=np.int32)
            return mask, class_ids
        else:
            # Call super class to return an empty mask
            return super(CocoDataset, self).load_mask(image_id)

    def image_reference(self, image_id):
        """Return a link to the image in the COCO Website."""
        info = self.image_info[image_id]
        if info["source"] == "coco":
            return "http://cocodataset.org/#explore?id={}".format(info["id"])
        else:
            super(CocoDataset, self).image_reference(image_id)

    # # The following two functions are from pycocotools with a few changes.

    # def annToRLE(self, ann, height, width):
    #     """
    #     Convert annotation which can be polygons, uncompressed RLE to RLE.
    #     :return: binary mask (numpy 2D array)
    #     """
    #     segm = ann['segmentation']
    #     if isinstance(segm, list):
    #         # polygon -- a single object might consist of multiple parts
    #         # we merge all parts into one mask rle code
    #         rles = maskUtils.frPyObjects(segm, height, width)
    #         rle = maskUtils.merge(rles)
    #     elif isinstance(segm['counts'], list):
    #         # uncompressed RLE
    #         rle = maskUtils.frPyObjects(segm, height, width)
    #     else:
    #         # rle
    #         rle = ann['segmentation']
    #     return rle

    # def annToMask(self, ann, height, width):
    #     """
    #     Convert annotation which can be polygons, uncompressed RLE, or RLE to binary mask.
    #     :return: binary mask (numpy 2D array)
    #     """
    #     rle = self.annToRLE(ann, height, width)
    #     m = maskUtils.decode(rle)
    #     return m
    
    def load_image(self, image_id):
      image_info = self.image_info[image_id]
      path = image_info.get('path')
      # print(image_info)
      # print(path)
      img = imread(path)[:,:,:3]
      img = resize(img, (config.IMAGE_SHAPE[0], config.IMAGE_SHAPE[1]), mode='constant', preserve_range=True)
    
      return img, image_info

      
  # def load_image(self, image_id):
  #     """Load the specified image and return a [H,W,3] Numpy array.
  #     """
  #     # Load image
  #     image = skimage.io.imread(self.image_info[image_id]['path'])
  #     # If grayscale. Convert to RGB for consistency.
  #     if image.ndim != 3:
  #         image = skimage.color.gray2rgb(image)
  #     # If has an alpha channel, remove it for consistency
  #     if image.shape[-1] == 4:
  #         image = image[..., :3]
  #     return image



In [ ]:
# class ShapesDataset(utils.Dataset):
    
#     def load_shapes(self, mode):
        
#         # Add classes
#         self.add_class("shapes", 1, "nucleus")
        

#         if mode == "train":  
#             for n, id_ in enumerate(train_ids):
#                 if n < int(len(train_ids) * 0.9):
#                     path = TRAIN_PATH + id_
#                     img_path = path + '/images/'
#                     self.add_image("shapes", image_id=id_, path=img_path)
              
#         if mode == "val":   
#             for n, id_ in enumerate(train_ids):
#                 if n >= int(len(train_ids) * 0.9):
#                     path = TRAIN_PATH + id_
#                     img_path = path + '/images/'
#                     self.add_image("shapes", image_id=id_, path=img_path)      

#     def load_image(self, image_id):
        
#         info = self.image_info[image_id]
#         info = info.get("id")
       
#         path = TRAIN_PATH + info
#         img = imread(path + '/images/' + info + '.png')[:,:,:3]
#         img = resize(img, (config.IMAGE_SHAPE[0], config.IMAGE_SHAPE[1]), mode='constant', preserve_range=True)
       
#         return img

#     def image_reference(self, image_id):
#         """Return the shapes data of the image."""
#         info = self.image_info[image_id]
#         if info["source"] == "shapes":
#             return info["shapes"]
#         else:
#             super(self.__class__).image_reference(self, image_id)

#     def load_mask(self, image_id):
#         """Generate instance masks for shapes of the given image ID.
#         """
        
#         info = self.image_info[image_id]
#         info = info.get("id")
#         path = TRAIN_PATH + info
#         number_of_masks = len(next(os.walk(path + '/masks/'))[2])
#         mask = np.zeros([config.IMAGE_SHAPE[0], config.IMAGE_SHAPE[1], number_of_masks], dtype=np.uint8)
#         iterator = 0
#         for mask_file in next(os.walk(path + '/masks/'))[2]:
#             mask_ = imread(path + '/masks/' + mask_file)
#             mask_ = resize(mask_, (config.IMAGE_SHAPE[0], config.IMAGE_SHAPE[1]), mode='constant', preserve_range=True)
#             mask[:, :, iterator] = mask_
#             iterator += 1
#             # Handle occlusions
#         occlusion = np.logical_not(mask[:, :, -1]).astype(np.uint8)
#         for i in range(number_of_masks-2, -1, -1):
#             mask[:, :, i] = mask[:, :, i] * occlusion
#             occlusion = np.logical_and(occlusion, np.logical_not(mask[:, :, i]))
            
#         # Map class names to class IDs.
#         class_ids = np.ones((number_of_masks,), dtype=int)
        
#         return mask, class_ids.astype(np.int32)
         
       
        

In [ ]:
# Training dataset
dataset_train = ShipsDataset()
dataset_train.load_coco(DATA_DIR, DATA_TYPE)
dataset_train.prepare()

# Validation dataset
dataset_val = ShipsDataset()
dataset_val.load_coco(DATA_DIR, DATA_TYPE)
dataset_val.prepare()

drive/MyDrive/HRSID/HRSID_png/
train_test2017
loading annotations into memory...
Done (t=0.95s)
creating index...
index created!
drive/MyDrive/HRSID/HRSID_png//images/
drive/MyDrive/HRSID/HRSID_png/
train_test2017
loading annotations into memory...
Done (t=0.56s)
creating index...
index created!
drive/MyDrive/HRSID/HRSID_png//images/


In [ ]:
# eval = EvalImage(dataset_train,model,config)
# # eval.evaluate_model()

In [ ]:
# Load and display random samples
image_ids = np.random.choice(dataset_train.image_ids, 4)
for image_id in image_ids:
    image, image_info = dataset_train.load_image(image_id)
    print(image_info['file_name'])
    mask, class_ids = dataset_train.load_mask(image_id)
    visualize.display_top_masks(image_info, image, mask, class_ids, dataset_train.class_names, limit=1)

drive/MyDrive/HRSID/HRSID_png//images/P0123_4800_5600_5400_6200.png
{'id': 5140, 'source': 'coco', 'path': 'drive/MyDrive/HRSID/HRSID_png//images/P0123_4800_5600_5400_6200.png', 'width': 800, 'height': 800, 'annotations': [{'id': 15452, 'image_id': 5140, 'category_id': 1, 'segmentation': [[43.0, 747.0, 42.0, 748.0, 41.0, 748.0, 40.0, 749.0, 39.0, 750.0, 38.0, 751.0, 38.0, 752.0, 39.0, 753.0, 39.0, 754.0, 39.0, 755.0, 40.0, 756.0, 40.0, 757.0, 40.0, 758.0, 41.0, 759.0, 42.0, 760.0, 42.0, 761.0, 43.0, 762.0, 43.0, 763.0, 44.0, 764.0, 44.0, 765.0, 44.0, 766.0, 45.0, 767.0, 45.0, 768.0, 45.0, 769.0, 45.0, 770.0, 46.0, 771.0, 46.0, 772.0, 46.0, 773.0, 47.0, 774.0, 47.0, 775.0, 47.0, 776.0, 48.0, 777.0, 48.0, 778.0, 48.0, 779.0, 48.0, 780.0, 49.0, 781.0, 49.0, 782.0, 49.0, 783.0, 50.0, 784.0, 50.0, 785.0, 50.0, 786.0, 51.0, 787.0, 51.0, 788.0, 51.0, 789.0, 52.0, 790.0, 52.0, 791.0, 52.0, 792.0, 53.0, 793.0, 53.0, 794.0, 54.0, 795.0, 55.0, 796.0, 56.0, 797.0, 57.0, 798.0, 58.0, 798.0, 59.0, 7

NameError: ignored

## Create Model

In [ ]:
# Create model in training mode
model = modellib.MaskRCNN(mode="training", config=config,
                          model_dir=MODEL_DIR)

In [ ]:
# you need to wrap the loss function with an empty lamda, when adding to the model
# model.keras_model.metrics_tensors = [] # 


# # Tensorflow board
# logdir = os.path.join(
#     "logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
# tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

# # load weights (mscoco) and exclude the output layers
# model.load_weights('mask_rcnn_coco.h5',
#                    by_name=True,
#                    exclude=[
#                        "mrcnn_class_logits", "mrcnn_bbox_fc", "mrcnn_bbox",
#                        "mrcnn_mask"
#                    ])

In [ ]:
# Which weights to start with?
init_with = "coco"  # imagenet, coco, or last

if init_with == "imagenet":
    model.load_weights(model.get_imagenet_weights(), by_name=True)
elif init_with == "coco":
    # Load weights trained on MS COCO, but skip layers that
    # are different due to the different number of classes
    # See README for instructions to download the COCO weights
    model.load_weights(COCO_MODEL_PATH, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                                "mrcnn_bbox", "mrcnn_mask"])
elif init_with == "last":
    # Load the last model you trained and continue training
    model.load_weights(model.find_last()[1], by_name=True)

## Training

Train in two stages:
1. Only the heads. Here we're freezing all the backbone layers and training only the randomly initialized layers (i.e. the ones that we didn't use pre-trained weights from MS COCO). To train only the head layers, pass `layers='heads'` to the `train()` function.

2. Fine-tune all layers. For this simple example it's not necessary, but we're including it to show the process. Simply pass `layers="all` to train all layers.

In [ ]:
# Train the head branches
# Passing layers="heads" freezes all layers except the head
# layers. You can also pass a regular expression to select
# which layers to train by name pattern.

model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE, 
            epochs=1, 
            layers='heads')



# checkpoint_filepath = '/tmp/checkpoint'
# model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
#     filepath=checkpoint_filepath,
#     save_weights_only=False,
#     monitor='val_mrcnn_mask_loss',
#     mode='min',
#     save_best_only=True)

# model.train(dataset_train, dataset_val,
#             learning_rate=config.LEARNING_RATE, 
#             epochs=1, 
#             layers='heads',
#             custom_callbacks=[model_checkpoint_callback])



# mean_average_precision_callback = modellib.MeanAveragePrecisionCallback(model,
# model, dataset_val, calculate_map_at_every_X_epoch=3, verbose=1)

# model.train(dataset_train, dataset_val,
#             learning_rate=config.LEARNING_RATE,
#             epochs=100,
#             layers='heads')

In [ ]:
# Fine tune all layers
# Passing layers="all" trains all layers. You can also 
# pass a regular expression to select which layers to
# train by name pattern.
"""model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE / 10,
            epochs=2, 
            layers="all")"""

In [ ]:
# Save weights
# Typically not needed because callbacks save after every epoch
# Uncomment to save manually
model_path = os.path.join(MODEL_DIR, "mask_rcnn_shapes3.h5")
model.keras_model.save_weights(model_path)

## Detection

In [ ]:
class InferenceConfig(ShapesConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig()

# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=MODEL_DIR)

# Get path to saved weights
# Either set a specific path or find last trained weights
model_path = os.path.join(ROOT_DIR, "logs/mask_rcnn_shapes3.h5")
# model_path = model.find_last()[1]

# Load trained weights (fill in path to trained weights here)
assert model_path != "", "Provide path to trained weights"
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

In [ ]:
# Test on a random image
image_id = random.choice(dataset_val.image_ids)
original_image, image_meta, gt_class_id, gt_bbox, gt_mask =\
    modellib.load_image_gt(dataset_val, inference_config, 
                           image_id)

log("original_image", original_image)
log("image_meta", image_meta)
log("gt_class_id", gt_class_id)
log("gt_bbox", gt_bbox)
log("gt_mask", gt_mask)

# Remove alpha channel, if it has one
if original_image.shape[-1] == 4:
    original_image = original_image[..., :3]

# visualize.display_images([original_image]+[mask[:,:,i] for i in range(min(mask.shape[-1], 7))])

visualize.display_instances(original_image, gt_bbox, gt_mask, gt_class_id, 
                            dataset_train.class_names, figsize=(8, 8))



# # image_ids = np.random.choice(dataset_val.image_ids,3)
# for image_id in dataset_val.image_ids:
# # for image_id in image_ids:
#     original_image, image_meta, gt_class_id, gt_bbox, gt_mask =\
#         modellib.load_image_gt(dataset_val, inference_config, 
#                                image_id)

#     log("original_image", original_image)
#     log("image_meta", image_meta)
#     log("gt_class_id", gt_bbox)
#     log("gt_bbox", gt_bbox)
#     log("gt_mask", gt_mask)

#     visualize.display_instances(original_image, gt_bbox, gt_mask, gt_class_id, dataset_train.class_names, figsize=(8, 8))

#     # results = model.detect([original_image], verbose=1)

#     # r = results[0]
#     # visualize.display_instances(original_image, r['rois'], r['masks'], r['class_ids'], 
#     #                             dataset_val.class_names, r['scores'], ax=get_ax())

In [ ]:
# Test on a random image
image_id = random.choice(dataset_val.image_ids)
original_image, image_meta, gt_class_id, gt_bbox, gt_mask =\
    modellib.load_image_gt(dataset_val, inference_config, 
                           image_id, augmentation=False)

log("original_image", original_image)
log("image_meta", image_meta)
log("gt_class_id", gt_class_id)
log("gt_bbox", gt_bbox)
log("gt_mask", gt_mask)

# Remove alpha channel, if it has one
if original_image.shape[-1] == 4:
    original_image = original_image[..., :3]

visualize.display_instances(original_image, gt_bbox, gt_mask, gt_class_id, 
                            dataset_train.class_names, figsize=(8, 8))

In [ ]:
# Test on a random image
image_id = random.choice(dataset_val.image_ids)
original_image, image_meta, gt_class_id, gt_bbox, gt_mask =\
    modellib.load_image_gt(dataset_val, inference_config, 
                           image_id, augmentation=False)

log("original_image", original_image)
log("image_meta", image_meta)
log("gt_class_id", gt_class_id)
log("gt_bbox", gt_bbox)
log("gt_mask", gt_mask)

# Remove alpha channel, if it has one
if original_image.shape[-1] == 4:
    original_image = original_image[..., :3]

visualize.display_instances(original_image, gt_bbox, gt_mask, gt_class_id, 
                            dataset_train.class_names, figsize=(8, 8))

In [ ]:
# Test on a random image
image_id = random.choice(dataset_val.image_ids)
original_image, image_meta, gt_class_id, gt_bbox, gt_mask =\
    modellib.load_image_gt(dataset_val, inference_config, 
                           image_id)

log("original_image", original_image)
log("image_meta", image_meta)
log("gt_class_id", gt_class_id)
log("gt_bbox", gt_bbox)
log("gt_mask", gt_mask)

visualize.display_instances(original_image, gt_bbox, gt_mask, gt_class_id, 
                            dataset_train.class_names, figsize=(8, 8))

In [ ]:
results = model.detect([original_image], verbose=1)

r = results[0]
visualize.display_instances(original_image, r['rois'], r['masks'], r['class_ids'], 
                            dataset_val.class_names, r['scores'], ax=get_ax())

In [ ]:
# Compute VOC-Style mAP @ IoU=0.5
# Running on 30 images. Increase for better accuracy.
image_ids = np.random.choice(dataset_val.image_ids, 10) # 30
APs = []
for image_id in image_ids:
    # Load image and ground truth data
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset_val, inference_config,
                               image_id)
    molded_images = np.expand_dims(modellib.mold_image(image, inference_config), 1)
    # Run object detection
    results = model.detect([image], verbose=1)
    r = results[0]
    # Compute AP
    AP, precisions, recalls, overlaps =\
        utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                         r["rois"], r["class_ids"], r["scores"], r['masks'])
    APs.append(AP)
    
print("mAP: ", np.mean(APs))

In [ ]:
X_test = np.zeros((len(test_ids), config.IMAGE_SHAPE[0], config.IMAGE_SHAPE[1], 3), dtype=np.uint8)
sizes_test = []
_test_ids = []

print('Getting and resizing test images ... ')
#sys.stdout.flush()
for n, id_ in enumerate(test_ids):
    _test_ids.append([id_])
    path = TEST_PATH + id_
    img = imread(path + '/images/' + id_ + '.png')[:,:,:3]
    sizes_test.append([img.shape[0], img.shape[1]])
    img = resize(img, (config.IMAGE_SHAPE[0], config.IMAGE_SHAPE[1]), mode='constant', preserve_range=True)
    X_test[n] = img

In [ ]:
print("checking a test image with masks ...")
results = model.detect([X_test[7]], verbose=1)

r = results[0]
visualize.display_instances(X_test[7], r['rois'], r['masks'], r['class_ids'], 
                            dataset_val.class_names, r['scores'], ax=get_ax())

In [ ]:

# # load photograph
# img = load_img('ship.jpg')
# img = img_to_array(img)

results = model.detect([original_image], verbose=1)

r = results[0]
visualize.display_instances(original_image, r['rois'], r['masks'], r['class_ids'], 
                            dataset_val.class_names, r['scores'], ax=get_ax())

## Evaluation

In [ ]:
# Compute VOC-Style mAP @ IoU=0.5
# Running on 30 images. Increase for better accuracy.
image_ids = np.random.choice(dataset_val.image_ids, 10)
APs = []
print(image_ids)
for image_id in image_ids:
    # Load image and ground truth data
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset_val, inference_config,
                               image_id)
    molded_images = np.expand_dims(modellib.mold_image(image, inference_config), 0)
    # Run object detection
    results = model.detect([image], verbose=0)
    r = results[0]
    # Compute AP
    AP, precisions, recalls, overlaps =\
        utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                         r["rois"], r["class_ids"], r["scores"], r['masks'])
    APs.append(AP)
print(APs)
    
print("mAP: ", np.mean(APs))

In [ ]:

X_test = np.zeros((len(test_ids), config.IMAGE_SHAPE[0], config.IMAGE_SHAPE[1], 3), dtype=np.uint8)
sizes_test = []
_test_ids = []

print('Getting and resizing test images ... ')
#sys.stdout.flush()
for n, id_ in enumerate(test_ids):
    _test_ids.append([id_])
    path = TEST_PATH + id_
    img = imread(path + '/images/' + id_ + '.png')[:,:,:3]
    sizes_test.append([img.shape[0], img.shape[1]])
    img = resize(img, (config.IMAGE_SHAPE[0], config.IMAGE_SHAPE[1]), mode='constant', preserve_range=True)
    X_test[n] = img

In [ ]:
print("checking a test image with masks ...")
results = model.detect([X_test[7]], verbose=1)

r = results[0]
visualize.display_instances(X_test[7], r['rois'], r['masks'], r['class_ids'], 
                            dataset_val.class_names, r['scores'], ax=get_ax())


In [ ]:
# Run-length encoding for submission
def rle_encoding(x):
    dots = np.where(x.T.flatten() == 1)[0]
    run_lengths = []
    prev = -2
    for b in dots:
        if (b>prev+1): run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    return run_lengths
  
  
def prob_to_rles(x):
   
    for i in range(0, len(x[1, 1, :])):
        y = x[:, :, i]
        yield rle_encoding(np.squeeze(y))

new_test_ids = []
rles = []

print(len(test_ids))

for n in range(0, len(test_ids)):

    results = model.detect([X_test[n]], verbose=0)
     
    r = results[0]

    number_of_masks = len(r['masks'][1, 1, :])
    
    pred_masks = np.zeros([sizes_test[n][0], sizes_test[n][1], number_of_masks], dtype=np.uint8)
   
    for m in range(0, number_of_masks):
        pred_mask_temp = r['masks'][:, :, m]
        pred_masks[:, :, m] = resize(np.squeeze(pred_mask_temp), (sizes_test[n][0], sizes_test[n][1]), mode='constant', preserve_range=True)
        
    # Handle occlusions
    occlusion = np.logical_not(pred_masks[:, :, -1]).astype(np.uint8)
    for i in range(number_of_masks-2, -1, -1):
        pred_masks[:, :, i] = pred_masks[:, :, i] * occlusion
        occlusion = np.logical_and(occlusion, np.logical_not(pred_masks[:, :, i]))
        
    rle = list(prob_to_rles(pred_masks))
        
            
    em = 0
    while (em < len(rle)):
          if len(rle[em]) == 1:
              print ("mask has one pixel")
          if len(rle[em]) == 0:
              print ("mask is empty. deleting mask")
              del rle[em]
          else:
              em = em + 1
    
    
    k = 0
    
    for m in range(0, len(rle)):
        while (k < len(rle[m]) ):
            if rle[m][k] + rle[m][k+1] >= sizes_test[n][0] * sizes_test[n][1]:
                print("Index was outside the bounds of the array.    index = ", rle[m][k] + rle[m][k+1], "    array size = ", sizes_test[n][0] * sizes_test[n][1] )
                rle[m][k+1] = sizes_test[n][0] * sizes_test[n][1] - rle[m][k] - 1
                #print("run length fixed ")
                print("run length fixed.    index = ", rle[m][k] + rle[m][k+1], "    array size = ", sizes_test[n][0] * sizes_test[n][1] )
                if rle[m][k+1] < 0:
                    print("run lngth is negative. deleting the index and run length   ")
                    del rle[m][k]
                    del rle[m][k]
                if rle[m][k] < 0:
                    print("index is negative. deleting the index and run length   ")
                    del rle[m][k]
                    del rle[m][k]
                    
           
                    
                 
                    
                #check that pixels are ordered 
                if rle[m][k] > rle[m][k+2]:
                   print ("pixel not ordered")
                
                #check that a pixel is not duplicated
                if rle[m][k] + rle[m][k+1] >= rle[m][k+2]:
                   print ("pixel duplicated")
                
            k = k + 2
    k = 0           
    for m in range(0, len(rle)):
        
        while (k < len(rle[m]) ):
            if rle[m][k] + rle[m][k+1] >= sizes_test[n][0] * sizes_test[n][1]:
                print("Index was outside the bounds of the array.    index = ", rle[m][k] + rle[m][k+1], "    array size = ", sizes_test[n][0] * sizes_test[n][1] )
            k = k + 2   
    
    rles.extend(rle)
    new_test_ids.extend(_test_ids[n] * len(rle))


# Create submission DataFrame

sub = pd.DataFrame()
sub['ImageId'] = new_test_ids
sub['EncodedPixels'] = pd.Series(rles).apply(lambda x: ' '.join(str(y) for y in x))
sub.to_csv('sub-dsbowl2018.csv', index=False)

    

In [ ]:
files.download('sub-dsbowl2018.csv')